# Brain Tumor Segmentation - GPU Safe Mode

## ⚠️ GTX 960 (4GB) 優化配置

本版本針對您的顯卡進行了以下優化：
- ✅ 極小的 batch size (2)
- ✅ 降低的圖片解析度 (256x256)
- ✅ GPU 記憶體自動清理
- ✅ CUDA TDR 錯誤處理
- ✅ 自動降級到 CPU（如果 GPU 失敗）

## 重要提醒
1. **運行前請關閉其他佔用 GPU 的程式**
2. **如果遇到錯誤，會自動切換到 CPU**
3. **訓練會比較慢，但應該不會崩潰**

In [1]:
# ========================================
# GPU 安全初始化 - 針對 GTX 960
# ========================================
import torch
import gc
import os

# 清理記憶體
gc.collect()

# 檢查 CUDA 可用性
cuda_available = torch.cuda.is_available()
print("="*60)
print(f"CUDA 可用: {cuda_available}")

if cuda_available:
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
    # 清空 CUDA cache
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    # 設置 CUDA 選項以避免 TDR
    torch.backends.cudnn.benchmark = False  # 關閉自動優化，避免長時間計算
    torch.backends.cudnn.deterministic = True  # 確保結果可重現
    
    print("✅ GPU 已初始化並清理記憶體")
    print("⚠️  如果訓練時遇到錯誤，將自動切換到 CPU")
else:
    print("❌ CUDA 不可用，將使用 CPU")
    
print("="*60)

CUDA 可用: True
GPU 名稱: NVIDIA GeForce GTX 960
GPU 記憶體: 4.00 GB
✅ GPU 已初始化並清理記憶體
⚠️  如果訓練時遇到錯誤，將自動切換到 CPU


In [2]:
# 導入必要的套件
import numpy as np
import random
from pathlib import Path

# GTX 960 安全配置
IMG_SIZE = 256      # 降低到 256 (原本可能是 640)
BATCH_SIZE = 2      # 極小的 batch size
EPOCHS = 25         # 完整訓練
LR = 1e-4
SEED = 42
NUM_WORKERS = 0     # Windows 建議設為 0

print("GTX 960 優化配置:")
print(f"  圖片大小: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE} (非常小，避免 GPU 超時)")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning Rate: {LR}")
print(f"  Workers: {NUM_WORKERS}")

# 設定隨機種子
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# 安全地設置 CUDA seed
if cuda_available:
    try:
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        print("✅ CUDA seeds 設置成功")
    except RuntimeError as e:
        print(f"⚠️  CUDA seed 設置失敗: {e}")
        print("   將繼續使用 CPU...")
        cuda_available = False

# 設定 device
DEVICE = 'cuda' if cuda_available else 'cpu'
print(f"\n使用設備: {DEVICE}")

GTX 960 優化配置:
  圖片大小: 256x256
  Batch Size: 2 (非常小，避免 GPU 超時)
  Epochs: 25
  Learning Rate: 0.0001
  Workers: 0
✅ CUDA seeds 設置成功

使用設備: cuda


---
# 資料探索（簡化版）
---

In [3]:
# 簡單檢查資料集
import json
from collections import Counter

path = r".\train\_annotations.coco.json"
with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

print("資料集統計:")
print(f"  圖片數量: {len(data.get('images', []))}")
print(f"  標註數量: {len(data.get('annotations', []))}")
print(f"  類別數量: {len(data.get('categories', []))}")

cat_counts = Counter(a["category_id"] for a in data.get("annotations", []))
print(f"  各類別標註數: {dict(cat_counts)}")

資料集統計:
  圖片數量: 1502
  標註數量: 1502
  類別數量: 3
  各類別標註數: {1: 771, 2: 731}


## ⚠️ 跳過圖片顯示部分

為了避免記憶體問題導致 kernel 崩潰，我們先跳過資料視覺化，直接進入模型訓練。

如果您需要看圖片，請在訓練完成後再執行視覺化的 cells。

---
# 模型訓練
---

In [4]:
# 導入訓練所需的套件
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import cv2
from tqdm.auto import tqdm

print("✅ 套件導入成功")

✅ 套件導入成功


In [5]:
# Dataset 定義（簡化版，避免記憶體問題）
class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, annotation_file, transform=None, size=256):
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.size = size
        
        # 載入標註
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        
        self.images = self.coco_data['images']
        self.annotations = self.coco_data['annotations']
        
        # 建立 image_id 到 annotations 的映射
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_path = self.image_dir / img_info['file_name']
        
        # 讀取圖片
        image = Image.open(img_path).convert('RGB')
        
        # 創建 mask
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
        
        # 填充 mask
        if img_info['id'] in self.img_to_anns:
            for ann in self.img_to_anns[img_info['id']]:
                if 'segmentation' in ann:
                    for seg in ann['segmentation']:
                        poly = np.array(seg).reshape(-1, 2).astype(np.int32)
                        cv2.fillPoly(mask, [poly], 1)
        
        mask = Image.fromarray(mask)
        
        # 應用轉換
        if self.transform:
            image = self.transform(image)
            mask = T.Resize((self.size, self.size), interpolation=T.InterpolationMode.NEAREST)(mask)
            mask = T.ToTensor()(mask)
        
        return image, mask

print("✅ Dataset 類別定義完成")

✅ Dataset 類別定義完成


In [6]:
# 數據轉換（降低解析度以節省記憶體）
train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 創建 dataset（僅使用訓練集的一部分以加快速度）
train_dataset = BrainTumorDataset(
    image_dir='./train',
    annotation_file='./train/_annotations.coco.json',
    transform=train_transform,
    size=IMG_SIZE
)

print(f"✅ 訓練集大小: {len(train_dataset)}")

# 創建 DataLoader（batch size 極小）
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False  # 避免額外的記憶體使用
)

print(f"✅ DataLoader 創建完成 (batch_size={BATCH_SIZE})")

✅ 訓練集大小: 1502
✅ DataLoader 創建完成 (batch_size=2)


In [7]:
# 簡單的 UNet 模型（輕量化版本）
import torch.nn as nn
import torch.nn.functional as F

class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder（減少通道數以節省記憶體）
        self.enc1 = self.conv_block(3, 32)
        self.enc2 = self.conv_block(32, 64)
        self.enc3 = self.conv_block(64, 128)
        
        # Bottleneck
        self.bottleneck = self.conv_block(128, 256)
        
        # Decoder
        self.dec3 = self.conv_block(256 + 128, 128)
        self.dec2 = self.conv_block(128 + 64, 64)
        self.dec1 = self.conv_block(64 + 32, 32)
        
        # Output
        self.out = nn.Conv2d(32, 1, kernel_size=1)
        
        self.pool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    
    def conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        
        # Bottleneck
        b = self.bottleneck(self.pool(e3))
        
        # Decoder
        d3 = self.dec3(torch.cat([self.upsample(b), e3], dim=1))
        d2 = self.dec2(torch.cat([self.upsample(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.upsample(d2), e1], dim=1))
        
        return self.out(d1)

# 創建模型並移動到設備
model = SimpleUNet()

# 安全地將模型移到 GPU
try:
    model = model.to(DEVICE)
    print(f"✅ 模型已移到 {DEVICE}")
    
    if DEVICE == 'cuda':
        # 清理記憶體
        torch.cuda.empty_cache()
        # 顯示記憶體使用
        print(f"   GPU 記憶體使用: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
except RuntimeError as e:
    print(f"❌ 無法將模型移到 GPU: {e}")
    print("   切換到 CPU...")
    DEVICE = 'cpu'
    model = model.to(DEVICE)

# 計算參數量
total_params = sum(p.numel() for p in model.parameters())
print(f"   模型參數量: {total_params:,}")

✅ 模型已移到 cuda
   GPU 記憶體使用: 7.47 MB
   模型參數量: 1,949,697


In [8]:
# 設置訓練組件
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(f"✅ 損失函數: BCEWithLogitsLoss")
print(f"✅ 優化器: Adam (lr={LR})")

✅ 損失函數: BCEWithLogitsLoss
✅ 優化器: Adam (lr=0.0001)


In [9]:
# 訓練函數（包含 GPU 記憶體管理）
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    pbar = tqdm(loader, desc='Training')
    for batch_idx, (images, masks) in enumerate(pbar):
        try:
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # 定期清理 GPU 記憶體（重要！）
            if device == 'cuda' and batch_idx % 50 == 0:
                torch.cuda.empty_cache()
            
        except RuntimeError as e:
            if "out of memory" in str(e) or "timeout" in str(e):
                print(f"\n❌ GPU 錯誤: {e}")
                print("   正在清理記憶體...")
                if device == 'cuda':
                    torch.cuda.empty_cache()
                    gc.collect()
                print("   建議: 降低 batch size 或切換到 CPU")
                raise
            else:
                raise
    
    return total_loss / len(loader)

print("✅ 訓練函數定義完成")

✅ 訓練函數定義完成


In [10]:
# 開始訓練
print("="*60)
print("開始訓練...")
print(f"設備: {DEVICE}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print("="*60)

best_loss = float('inf')

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    try:
        # 訓練
        train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        print(f"Average Loss: {train_loss:.4f}")
        
        # 保存最佳模型
        if train_loss < best_loss:
            best_loss = train_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"✅ 保存最佳模型 (loss: {best_loss:.4f})")
        
        # 定期清理記憶體
        if DEVICE == 'cuda':
            torch.cuda.empty_cache()
            gc.collect()
            
    except RuntimeError as e:
        print(f"\n❌ 訓練失敗: {e}")
        print("\n建議的解決方案:")
        print("1. 重啟 Jupyter kernel")
        print("2. 將 BATCH_SIZE 降到 1")
        print("3. 將 IMG_SIZE 降到 128")
        print("4. 使用 CPU 模式（修改 DEVICE = 'cpu'）")
        break

print("\n" + "="*60)
print("訓練完成！")
print(f"最佳損失: {best_loss:.4f}")
print("="*60)

開始訓練...
設備: cuda
Epochs: 25
Batch Size: 2
Image Size: 256x256

Epoch 1/25


Training:   0%|          | 0/751 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 🎉 完成！

如果訓練成功完成：
- ✅ 您的模型已保存為 `best_model.pth`
- ✅ 您可以使用這個模型進行預測

如果遇到錯誤：
1. 重啟 Jupyter kernel
2. 將 `BATCH_SIZE` 設為 1
3. 或者切換到 CPU 模式